In [1]:
import pandas as pd
import numpy as np
import os 
import cv2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.text import Text
import string
import pickle
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score,classification_report
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.datasets import load_files
import os
from IPython.core.display import HTML, display
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#import easyocr

C:\Users\Faizan\AppData\Local\Temp\ipykernel_17628\675573221.py:29: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


# Import dataset

In [2]:
missing_val=["n.a.","?","NA","n/a","--"]
df= pd.read_csv("C://Users//Faizan//OneDrive - FAST National University//Pictures//final project//labels.csv",na_values=missing_val)
df.drop(['Unnamed: 0','text_ocr'],axis=1,inplace=True)
df

,image_name,text_corrected,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,neutral
...,...,...,...
6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,neutral
6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,neutral
6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,positive
6990,image_6991.jpg,When I have time is a fantasy. no one has time...,very_positive


# Drop null rows

In [3]:
arr=df['image_name']
df.drop(['image_name'],axis=1,inplace=True)
df.dropna(inplace=True)
df

,text_corrected,overall_sentiment
0,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,very_positive
1,The best of #10 YearChallenge! Completed in le...,very_positive
2,Sam Thorne @Strippin ( Follow Follow Saw every...,positive
3,10 Year Challenge - Sweet Dee Edition,positive
4,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,neutral
...,...,...
6987,Tuesday is Mardi Gras Wednesday is Valentine's...,neutral
6988,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,neutral
6989,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,positive
6990,When I have time is a fantasy. no one has time...,very_positive


In [4]:
labels = df['overall_sentiment']
l = []
# making the labels dataset have 3 classes only 
for i in labels:
    if(i == 'very_positive' or i == 'positive'):
        l.append(0)
    if(i == 'very_negative' or i == 'negative'):
        l.append(1)
    if(i == 'neutral'):    
        l.append(2)
df['overall_sentiment']=l


In [5]:
df

,text_corrected,overall_sentiment
0,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,0
1,The best of #10 YearChallenge! Completed in le...,0
2,Sam Thorne @Strippin ( Follow Follow Saw every...,0
3,10 Year Challenge - Sweet Dee Edition,0
4,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,2
...,...,...
6987,Tuesday is Mardi Gras Wednesday is Valentine's...,2
6988,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,2
6989,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,0
6990,When I have time is a fantasy. no one has time...,0


# Data Cleaning

In [6]:
import re
df['text_corrected'] = df['text_corrected'].str.replace('\s+[a-zA-Z]\s+', ' ', regex=True)   
#df['text_corrected'] = df['text_corrected'].str.replace(r'\^[a-zA-Z]\s+', ' ', regex=True)   
df['text_corrected'] = df['text_corrected'].str.replace(r'^b\s+', '', regex=True)
df['text_corrected'] = df['text_corrected'].str.replace('\W', ' ', regex=True)
#df['text_corrected'] = df['text_corrected'].str.replace('\d+', '')
#df['text_corrected'] = df['text_corrected'].str.replace('_', ' ')
df['text_corrected']=df['text_corrected'].str.lower()
df

,text_corrected,overall_sentiment
0,look there my friend lightyear now all sohalik...,0
1,the best of 10 yearchallenge completed in le...,0
2,sam thorne strippin follow follow saw every...,0
3,10 year challenge sweet dee edition,0
4,10 year challenge with no filter 47 hilarious ...,2
...,...,...
6987,tuesday is mardi gras wednesday is valentine s...,2
6988,must watch movies of 2017 iti chennai memes ma...,2
6989,less more talking planning soda junk food comp...,0
6990,when have time is fantasy no one has time unl...,0


In [7]:
stop_words = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]
len(stop_words)

1160

# Removing stopwords

In [8]:

df['text_corrected'] = df['text_corrected'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [9]:
sent=[]
comp=neg=pos=neu=[]
sent=df['text_corrected']
s=len(sent)
ind=[*range(0, s, 1)]
sent.index=ind
len(sent)

6987

# Sentiment Analysis

In [10]:

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
 

def sentiment_scores(sentence):
 

    sid_obj = SentimentIntensityAnalyzer()
 

    sentiment_dict = sid_obj.polarity_scores(sentence)
     
#    print("Overall sentiment dictionary is : ", sentiment_dict)
#     print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
#     print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
#     print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
    c=sentiment_dict['compound']
#    print(c)
    neg.append(sentiment_dict['neg'])
    neu.append(sentiment_dict['neu'])
    pos.append(sentiment_dict['pos'])
    comp.append(c)


for i in range(len(sent)):
    
    sentiment_scores(sent[i])


In [11]:
# len(comp)
#comp=comp[3::3]
new_neg=neg[::4]
#len(neg[::4])
new_neu=neu[1::4]
new_pos=pos[2::4]
#pos=pos[2::4]
#neg[0::4]

new_cmp=comp[3::4]


In [12]:
# label_encoding = preprocessing.LabelEncoder()
# df['overall_sentiment'] = label_encoding.fit_transform(df['overall_sentiment'].astype(str))
# #df = label_encoding.fit_transform(df.astype(str))
# df['overall_sentiment']

# Conversion of text through vectorizer

In [13]:
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
bag_of_words = vectorizer.fit_transform(df['text_corrected'])

In [14]:
# from sklearn.feature_extraction.text import TfidfTransformer
# tfidfconverter = TfidfTransformer()
# X = tfidfconverter.fit_transform(X).toarray()
# X

# Updating Numerical dataframe

In [15]:
df2=pd.DataFrame(bag_of_words.toarray(), columns=vectorizer.get_feature_names_out())
df2['pos']=new_pos
df2['neg']=new_neg
df2['neu']=new_neu
df2['comp']=new_cmp
df2.fillna(0,inplace=True)

In [16]:
df2

,00,000,08,10,100,11,12,13,14,15,...,young,youtube,zayn,zipmeme,zombie,zuckerberg,pos,neg,neu,comp
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.496,0.000,0.504,0.7096
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000,0.000,1.000,0.0000
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.113,0.000,0.887,0.4215
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.518,0.000,0.482,0.5106
4,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0.423,0.000,0.577,0.5574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6982,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.209,0.000,0.791,0.5719
6983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000,0.000,1.000,0.0000
6984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.393,0.351,0.255,0.7003
6985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000,0.000,1.000,0.0000


# Train_Test_split

In [17]:
X = df2
y = df.overall_sentiment
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.20) # 70% training and 30% test

In [18]:
from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
X_train= st_x.fit_transform(X_train)    
X_test= st_x.transform(X_test)   


# Decision Tree Classifier

In [19]:
clf = DecisionTreeClassifier(criterion='entropy',max_depth=2000,splitter='best', random_state=0 ,   min_samples_split=10,
min_samples_leaf=5)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print("F1 Score: ",f1_score(y_test, y_pred,average='weighted')) 
print(classification_report(y_test, y_pred))

Accuracy: 0.5329041487839771
F1 Score:  0.4925265275961244
              precision    recall  f1-score   support

           0       0.60      0.77      0.68       826
           1       0.11      0.03      0.05       129
           2       0.34      0.23      0.28       443

    accuracy                           0.53      1398
   macro avg       0.35      0.35      0.33      1398
weighted avg       0.47      0.53      0.49      1398



# Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs',random_state=100).fit(X_train, y_train)
p=clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, p))
print("F1 Score: ",f1_score(y_test, p,average='weighted')) 
print(classification_report(y_test, p))

Accuracy: 0.474964234620887
F1 Score:  0.46924901863060037
              precision    recall  f1-score   support

           0       0.60      0.64      0.62       826
           1       0.09      0.09      0.09       129
           2       0.32      0.28      0.30       443

    accuracy                           0.47      1398
   macro avg       0.34      0.34      0.34      1398
weighted avg       0.47      0.47      0.47      1398



C:\Users\Faizan\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# SVM 

In [21]:
from sklearn import svm
svc = svm.SVC(kernel='poly', C=1).fit(X_train, y_train)
p=svc.predict(X_test)
print('Accuracy:', accuracy_score(y_test, p))
print("F1 Score: ",f1_score(y_test, p,average='weighted')) 
print(classification_report(y_test, p))

Accuracy: 0.5636623748211731
F1 Score:  0.45170412803585613
              precision    recall  f1-score   support

           0       0.59      0.93      0.72       826
           1       0.21      0.02      0.04       129
           2       0.24      0.04      0.07       443

    accuracy                           0.56      1398
   macro avg       0.35      0.33      0.28      1398
weighted avg       0.44      0.56      0.45      1398



# KNN Model

In [22]:
from sklearn.neighbors import KNeighborsClassifier

clf1=KNeighborsClassifier(n_neighbors=5)
clf1.fit(X_train, y_train)
pred=clf1.predict(X_test)
clf1.score(X_test,y_test)
print('Accuracy:', accuracy_score(y_test, pred))

Accuracy: 0.5293276108726752


In [23]:
print('Accuracy:', accuracy_score(y_test, pred))
print("F1 Score: ",f1_score(y_test, pred,average='weighted')) 
print(classification_report(y_test, pred))
txt1_f1=f1_score(y_test, pred,average='weighted')

Accuracy: 0.5293276108726752
F1 Score:  0.4850959868224525
              precision    recall  f1-score   support

           0       0.59      0.77      0.67       826
           1       0.26      0.05      0.08       129
           2       0.32      0.21      0.25       443

    accuracy                           0.53      1398
   macro avg       0.39      0.34      0.34      1398
weighted avg       0.48      0.53      0.49      1398



In [24]:
filename = "C://Users/Faizan/OneDrive - FAST National University/Pictures/final project/KNN_txt.joblib"
joblib.dump(clf1, filename)

['C://Users/Faizan/OneDrive - FAST National University/Pictures/final project/KNN_txt.joblib']

# Naive Bayes 

In [25]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train);
pred=clf1.predict(X_test)
clf1.score(X_test,y_test)
print('Accuracy:', accuracy_score(y_test, pred))
txt2_f1=print("F1 Score: ",f1_score(y_test, pred,average='weighted')) 
print(classification_report(y_test, pred))
txt2_f1=f1_score(y_test, pred,average='weighted')

Accuracy: 0.5293276108726752
F1 Score:  0.4850959868224525
              precision    recall  f1-score   support

           0       0.59      0.77      0.67       826
           1       0.26      0.05      0.08       129
           2       0.32      0.21      0.25       443

    accuracy                           0.53      1398
   macro avg       0.39      0.34      0.34      1398
weighted avg       0.48      0.53      0.49      1398



In [26]:
filename = "C://Users/Faizan/OneDrive - FAST National University/Pictures/final project/Naive-Bayes_txt.joblib"
joblib.dump(clf1, filename)

['C://Users/Faizan/OneDrive - FAST National University/Pictures/final project/Naive-Bayes_txt.joblib']

In [27]:
from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(X_train)    
x_test= st_x.transform(X_test)   

# Random Forest

In [28]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  
y_pred= classifier.predict(x_test)  
classifier.score(x_test,y_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print("F1 Score: ",f1_score(y_test, y_pred,average='weighted')) 
print(classification_report(y_test, y_pred))
cm= metrics.confusion_matrix(y_test, y_pred)  
txt3_f1=f1_score(y_test, y_pred,average='weighted')

Accuracy: 0.5171673819742489
F1 Score:  0.47874878240784985
              precision    recall  f1-score   support

           0       0.59      0.75      0.66       826
           1       0.15      0.05      0.07       129
           2       0.32      0.22      0.26       443

    accuracy                           0.52      1398
   macro avg       0.35      0.34      0.33      1398
weighted avg       0.46      0.52      0.48      1398



In [29]:
cm

array([[620,  22, 184],
       [ 98,   6,  25],
       [334,  12,  97]], dtype=int64)

In [30]:
filename = "C://Users/Faizan/OneDrive - FAST National University/Pictures/final project/RandomForest_txt.joblib"
joblib.dump(classifier, filename)

['C://Users/Faizan/OneDrive - FAST National University/Pictures/final project/RandomForest_txt.joblib']

# Image Training

In [26]:
import re
import cv2
from skimage.io import imread,imshow
from skimage import color
path="C://Users//Faizan//OneDrive - FAST National University//Pictures//final project//images//"



# Converting images to array

In [27]:
new=df.index
ind=[]
ite=0
images=[]
for i in arr:

    try:
        image = cv2.imread(path+i)
        image = color.rgb2gray(image)
        size=(100, 100)
        resized_image_feature_vector = cv2.resize(image, size)
        resized_flattened_image_feature_vector = resized_image_feature_vector.flatten()
        images.append(resized_flattened_image_feature_vector)
        ind.append(ite)
        ite+=1
    except:
        continue

In [28]:
images


[array([0.39505098, 0.39505098, 0.39505098, ..., 0.29454353, 0.28719059,
        0.28579176]),
 array([1., 1., 1., ..., 1., 1., 1.]),
 array([0.19650995, 0.20351725, 0.18796471, ..., 0.49803922, 0.49803922,
        0.49803922]),
 array([0.4073948 , 0.39951536, 0.40996534, ..., 1.        , 1.        ,
        1.        ]),
 array([0.98210955, 0.98767765, 0.988052  , ..., 1.        , 1.        ,
        1.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.98746029, 0.98746029, 0.98746029, ..., 1.        , 1.        ,
        1.        ]),
 array([1.        , 1.        , 1.        , ..., 0.62518534, 0.61548392,
        0.32298011]),
 array([1., 1., 1., ..., 1., 1., 1.]),
 array([1.        , 1.        , 1.        , ..., 0.93305588, 0.95876615,
        0.94544737]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([1.        , 1.        , 1.        , ..., 0.02367239, 0.03806728,
        0.12829324]),
 array([1., 1., 1., ..., 1., 1., 1.]),
 array([1.        , 1.        , 1.        , .

In [29]:
df_img=pd.DataFrame(images,index =ind)
df_img

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.395051,0.395051,0.395051,0.395051,0.398973,0.404572,0.409728,0.412442,0.421424,0.421438,...,0.299238,0.294483,0.292865,0.297006,0.295316,0.291112,0.294151,0.294544,0.287191,0.285792
1,1.000000,1.000000,1.000000,1.000000,1.000000,0.991807,0.997736,0.978223,0.974177,0.956082,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,0.196510,0.203517,0.187965,0.187181,0.173795,0.181520,0.185442,0.185442,0.179484,0.996365,...,1.000000,1.000000,0.453134,0.464114,0.465996,0.498039,0.498039,0.498039,0.498039,0.498039
3,0.407395,0.399515,0.409965,0.414826,0.421570,0.426888,0.422952,0.402276,0.421573,0.426219,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,0.982110,0.987678,0.988052,0.988258,0.988258,0.978725,0.988755,0.982110,0.984874,0.982110,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6983,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6984,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6985,0.630458,0.630458,0.630458,0.628825,0.628825,0.628825,0.628825,0.630458,0.626905,0.630458,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6986,0.568002,0.594201,0.606275,0.616107,0.627451,0.639216,0.646746,0.655427,0.662745,0.674510,...,0.714976,0.720139,0.712714,0.700707,0.688533,0.685229,0.670765,0.658274,0.654902,0.639216


In [30]:
new_df=pd.DataFrame()
new_df['sentiment']=df['overall_sentiment']
new_df

,sentiment
0,0
1,0
2,0
3,0
4,2
...,...
6987,2
6988,2
6989,0
6990,0


# Equalizing values for datasets

In [31]:
l=[]
l1=[]
for i in ind:
    if i not in new:
#        print(i)
        l.append(i)
#         df_img.drop(df_img.i)
        
        
print("now next df")
for i in new:
    if i not in ind:
#        print(i)
        l1.append(i)
#         new_df.drop(new_df.i)

now next df


In [32]:
for i in l:
    df_img.drop(labels=i, inplace=True)

new_df = new_df.iloc[:-4 , :]

In [33]:
df_img.isnull().sum()

0       0
1       0
2       0
3       0
4       0
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Length: 10000, dtype: int64

In [34]:
new_df

,sentiment
0,0
1,0
2,0
3,0
4,2
...,...
6983,0
6984,0
6985,2
6986,0


# Train_test_split

In [35]:
X = df_img
y = new_df.sentiment
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.20,random_state=900,) # 70% training and 30% test

# Random Forest Classifier

In [36]:
from sklearn.preprocessing import StandardScaler
st_x= StandardScaler()    
x_train= st_x.fit_transform(X_train)    
x_test= st_x.transform(X_test)   

In [37]:
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import confusion_matrix  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  
y_pred= classifier.predict(x_test)  
classifier.score(x_test,y_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print("F1 Score: ",f1_score(y_test, y_pred,average='weighted')) 
print(classification_report(y_test, y_pred))
cm= confusion_matrix(y_test, y_pred) 
img1_f1=f1_score(y_test, y_pred,average='weighted')

Accuracy: 0.5490336435218325
F1 Score:  0.4774861872581987
              precision    recall  f1-score   support

           0       0.60      0.84      0.70       836
           1       0.33      0.01      0.03       139
           2       0.29      0.14      0.19       422

    accuracy                           0.55      1397
   macro avg       0.41      0.33      0.30      1397
weighted avg       0.48      0.55      0.48      1397



In [38]:
cm

array([[706,   2, 128],
       [119,   2,  18],
       [361,   2,  59]], dtype=int64)

In [ ]:
filename = "C://Users/Faizan/OneDrive - FAST National University/Pictures/final project/Random-Forest_img.joblib"
joblib.dump(classifier, filename)

In [39]:
from sklearn.neighbors import KNeighborsClassifier

clf1=KNeighborsClassifier(n_neighbors=5)
clf1.fit(X_train, y_train)
pred=clf1.predict(X_test)
clf1.score(X_test,y_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print("F1 Score: ",f1_score(y_test, y_pred,average='weighted')) 
print(classification_report(y_test, y_pred))
cm= confusion_matrix(y_test, y_pred) 
img2_f1=f1_score(y_test, y_pred,average='weighted')
cm

Accuracy: 0.5490336435218325
F1 Score:  0.4774861872581987
              precision    recall  f1-score   support

           0       0.60      0.84      0.70       836
           1       0.33      0.01      0.03       139
           2       0.29      0.14      0.19       422

    accuracy                           0.55      1397
   macro avg       0.41      0.33      0.30      1397
weighted avg       0.48      0.55      0.48      1397



array([[706,   2, 128],
       [119,   2,  18],
       [361,   2,  59]], dtype=int64)

In [ ]:
filename = "C://Users/Faizan/OneDrive - FAST National University/Pictures/final project/KNN_img.joblib"
joblib.dump(clf1, filename)

In [40]:
clf = DecisionTreeClassifier(criterion='entropy',max_depth=2000,splitter='best', random_state=0 ,   min_samples_split=10,
min_samples_leaf=5)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print("F1 Score: ",f1_score(y_test, y_pred,average='weighted')) 
print(classification_report(y_test, y_pred))
cm= confusion_matrix(y_test, y_pred) 
img3_f1=f1_score(y_test, y_pred,average='weighted')
cm

Accuracy: 0.4581245526127416
F1 Score:  0.4569188174400592
              precision    recall  f1-score   support

           0       0.59      0.59      0.59       836
           1       0.08      0.07      0.08       139
           2       0.31      0.32      0.31       422

    accuracy                           0.46      1397
   macro avg       0.33      0.33      0.33      1397
weighted avg       0.46      0.46      0.46      1397



array([[496,  79, 261],
       [ 90,  10,  39],
       [251,  37, 134]], dtype=int64)

In [84]:
filename = "C://Users/Faizan/OneDrive - FAST National University/Pictures/final project/Decision-Tree_img.joblib"
joblib.dump(clf, filename)
#pickle.dump(clf,open('/content/Decision-Tree_img/C://Users/Faizan/OneDrive - FAST National University/Pictures/final project'),'wb')

['C://Users/Faizan/OneDrive - FAST National University/Pictures/final project/Decision-Tree_img.joblib']

# Total F1-Score on all models

In [41]:
(txt1_f1+txt1_f1+txt1_f1+img1_f1+img2_f1+img1_f1)/6

0.4715890071555433

In [97]:
import pytesseract
import cv2
import matplotlib.pyplot as plt
from PIL import Image
pytesseract.pytesseract.tesseract_cmd =  r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [98]:
image = cv2.imread(path+arr[0])
string = pytesseract.image_to_string(image)

In [100]:
print(string)

ee

NOW ALL LU SOHALIK IKUTTREND PLAY |
THE 1OYEARS CHALLENGE AT FACEBOOK

eater

